In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../lstm_data.csv"))

# Any results you write to the current directory are saved as output.

In [8]:
train = pd.read_csv('lstm_data.csv', dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32}).values
len(train)

2627

In [10]:
train [0:2627, 4] .mean(axis=0)


0.22725542443852303

In [17]:
rows = 150
segments = int(np.floor(train.shape[0] / rows))
print('train.shape',train.shape)
segments

train.shape (2627, 6)


17

In [18]:
def extract_features(z):
     return np.c_[z.mean(axis=0), 
                  z.std(axis=0),
                  z.max(axis=0),
                  z.min(axis=0),
                #  np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=0)) .reshape(1,4)
                 ]

In [19]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
       
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    # ORIGINAL: I changed this becuase I got an No OpKernel was registered to support Op 'CudnnRNN' error
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    # MY CHANGE: This doesn't fix things, I get the same errors
    temp = (x[(int(last_index) - n_steps * step_length):int(last_index)].reshape(n_steps,step_length ).astype(np.float32) - 5 ) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp [ -n_steps // 10:]),
                 extract_features(temp [ -n_steps // 100:])]